In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import xarray as xr
import numpy as np
import pandas as pd

In [4]:
xr.set_options(keep_attrs=True)

In [5]:
import ecephys_analyses as ea 
from ecephys import utils
from ecephys.signal.xarray_utils import filter_dataset_by_hypnogram

In [6]:
def ds2df(ds):
    s = ds.to_dataframe().stack()
    s.index.names = s.index.names[:-1] + ['region']
    return s.to_frame(name='power')

In [7]:
subject = "Allan"
experiment = "sleep-homeostasis"
bsl_condition = "recovery-sleep-circadian-match"
rec_condition = "recovery-sleep"
states = ["N1", "N2"]
duration = '02:00:00'
light_period_end = '21:00:00'
f_range = (0.5, 4)

In [8]:
rec_hyp = ea.load_hypnogram(subject, experiment, rec_condition)
rec_hyp = rec_hyp.keep_states(states).keep_first(duration)

rec_pow = ea.load_power(subject, experiment, rec_condition, "spg.nc").sel(frequency=slice(*f_range)).sum(dim='frequency')
rec_pow = filter_dataset_by_hypnogram(rec_pow, rec_hyp,).swap_dims({'time': 'datetime'}).drop_vars(['time', 'timedelta'])

In [9]:
bsl_hyp = ea.load_hypnogram(subject, experiment, bsl_condition)
bsl_hyp = bsl_hyp.keep_states(states).keep_between(rec_hyp.start_time.min().strftime("%H:%M:%S"), light_period_end).keep_first(duration)

bsl_pow = ea.load_power(subject, experiment, bsl_condition, "spg.nc").sel(frequency=slice(*f_range)).sum(dim='frequency')
bsl_pow = filter_dataset_by_hypnogram(bsl_pow, bsl_hyp).swap_dims({'time': 'datetime'}).drop_vars(['time', 'timedelta'])

In [10]:
nrem_pow = pd.concat([ds2df(bsl_pow).assign(condition="baseline"), ds2df(rec_pow).assign(condition="recovery")])

In [11]:
nrem_pow

power condition
datetime                      region                              
2021-03-01 15:01:46.000138841 mpta_wm_ref  158116.604710  baseline
                              sr_wm_ref    268833.530966  baseline
2021-03-01 15:01:49.000139731 mpta_wm_ref  318828.737459  baseline
                              sr_wm_ref    239860.983292  baseline
2021-03-01 15:01:52.000140621 mpta_wm_ref  342869.221709  baseline
...                                                  ...       ...
2021-03-02 18:09:16.999961706 sr_wm_ref    554251.950719  recovery
2021-03-02 18:09:37.999967937 mpta_wm_ref  280896.170848  recovery
                              sr_wm_ref    398208.624081  recovery
2021-03-02 18:09:40.999968827 mpta_wm_ref  391427.489301  recovery
                              sr_wm_ref    484118.821511  recovery

[9550 rows x 2 columns]

In [12]:
g = sns.displot(data=nrem_pow, x='power', hue='condition', col='region', kind='hist')
g.set(xlabel='Delta (0.5 - 4Hz) PSD (uV^2/Hz)')
g.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
nrem_pow_log = nrem_pow.copy()
nrem_pow_log['power'] = np.log(nrem_pow.power)

g = sns.displot(data=nrem_pow_log, x='power', hue='condition', col='region', kind='hist')
g.set(xlabel='Delta (0.5 - 4Hz) PSD (uV^2/Hz)')
g.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
nrem_pow_sqrt = nrem_pow.copy()
nrem_pow_sqrt['power'] = np.sqrt(nrem_pow.power)

g = sns.displot(data=nrem_pow_sqrt, x='power', hue='condition', col='region', kind='hist')
g.set(xlabel='Delta (0.5 - 4Hz) PSD (uV^2/Hz)')
g.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
def difference_of_means(df):
    means = df.groupby(['condition', 'region']).mean()
    return means.xs('recovery') - means.xs('baseline')

In [16]:
observed = difference_of_means(nrem_pow)

n_permutations = 10000
permutations = list()
permuted = nrem_pow.copy()
for i in range(n_permutations):
    np.random.shuffle(permuted.condition.values)
    permutations.append(difference_of_means(permuted))
    
df = pd.concat(permutations)

In [17]:
g = sns.displot(data=df, x='power', col='region', kind='hist')
g.set(xlabel='Difference of means (SD - BL)')
g.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
cx = df.xs('mpta_wm_ref').sort_values('power')
hc = df.xs('sr_wm_ref').sort_values('power')

In [19]:
pcx = np.sum(cx.power.values >= observed.xs('mpta_wm_ref').values)

In [20]:
phc = np.sum(hc.power.values >= observed.xs('sr_wm_ref').values)

In [21]:
print(f"Subject: {subject}")
print('Observed differences, Mean SD - Mean BSL:\n')
print(observed.to_markdown())
print('\n')
print('Shuffled differences >= observed:')
print(f"Cortex: {1 + pcx} / {n_permutations}")
print(f"Hippocampus: {1 + phc} / {n_permutations}")

Subject: Allan
Observed differences, Mean SD - Mean BSL:

| region      |    power |
|:------------|---------:|
| mpta_wm_ref | 620822   |
| sr_wm_ref   |  96984.4 |


Shuffled differences >= observed:
Cortex: 1 / 10000
Hippocampus: 1 / 10000
